sequentially split the trace

In [ ]:
import pandas as pd
import numpy as np


nums = ['050', '100', '200', '250', '300']
for num in nums:
    path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_pod_list_cpu' + num + '.csv'
    df = pd.read_csv(path)
    df['num_gpu'] = 0
    df['gpu_milli'] = 0
    df['gpu_spec'] = ''
    running_df = df
    # Filter the dataframe
    # running_df = df[df['pod_phase'] == 'Running']
    # running_df = running_df[running_df['qos'] == 'LS']
    running_df = running_df[running_df['num_gpu'] == 0]
    
    tenants = [1, 2, 3, 5, 10, 15, 20]
    for tenant in tenants:
        # Split the dataframe into parts
        dfs = np.array_split(running_df, tenant)
        # Save each part to a separate CSV file
        for i, df_part in enumerate(dfs, 1):
            out_file = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/fabing_cpu_{}_{}_{}.csv'.format(num, tenant, i)
            print('data/fabing_cpu_{}_{}_{}'.format(num, tenant, i))
            df_part.to_csv(out_file.format(i), index=False)


split the table

In [ ]:
# Filter the dataframe
running_df = df[df['pod_phase'] == 'Running']

# running_df = running_df[running_df['qos'] == 'LS']
running_df = running_df[running_df['num_gpu'] == 0]

# Save to a new CSV file
running_df.to_csv('/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/fabing_running_all.csv', index=False)

# Split the dataframe into 3 equal parts
dfs = np.array_split(running_df, 3)

# Save each part to a separate CSV file
for i, df_part in enumerate(dfs, 1):
    df_part.to_csv(f'/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/fabing_running_part{i}_all.csv', index=False)

Output the number of used nodes

In [ ]:
import pandas as pd

path = '/home/fabing/projects/kubernetes-scheduler-simulator/experiments/2023_1123/fabing_cpu_300_20_13/05-BestFit/1.3/42/analysis_allo.csv'
df = pd.read_csv(path)

df.columns = [x.split('-')[-1] for x in df.columns]
dfd = df.to_dict(orient="list")
print(max(df['used_nodes']))
total_gpu_num = df.total_gpus.values[0]
df['arrive_ratio'] = df.arrived_gpu_milli / total_gpu_num / 10
df['arrive_ratio'] = df['arrive_ratio'].apply(lambda x: round(x, 0))
df['alloc_ratio'] = df.used_gpu_milli / total_gpu_num / 10
df['alloc_ratio'] = df['alloc_ratio'].apply(lambda x: round(x, 2))

Further processing of analysis results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


out_file = '/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_allo_discrete_sum_use_nodes_uniform.csv'
# Load the dataframe from the CSV file
# df = pd.read_csv('/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_allo_discrete.csv')
df = pd.read_csv('/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_allo_discrete.csv')
df.drop(columns=['sc_policy', 'tune', 'seed'], inplace=True)

# Group by 'workload' and calculate the mean of the other columns
df = df.groupby('workload').mean()
# Reset the index
df = df.reset_index()
# Extract the prefix of the 'workload' column
df['workload_prefix'] = df['workload'].str.rsplit('_', n=1).str[0]

# Calculate the average 'use_nodes' for each 'workload_prefix'
average_use_nodes = df.groupby('workload_prefix')['use_nodes'].sum()
average_use_nodes.to_csv(out_file)
df = pd.read_csv(out_file)
# Split the 'workload_prefix' into four parts
df[['prefix', 'cpu', 'first_number', 'second_number']] = df['workload_prefix'].str.split('_', n=3, expand=True)

# Convert the 'first_number' and 'second_number' columns to numeric types
df[['first_number', 'second_number']] = df[['first_number', 'second_number']].apply(pd.to_numeric)

# Group by 'first_number' and calculate the mean of 'use_nodes'
grouped_df = df.groupby('first_number')['use_nodes'].mean()

# Sort by 'second_number'
sorted_df = df.sort_values('first_number')

# Draw bar charts
for i in sorted_df['first_number'].unique():
    # Filter the dataframe
    filtered_df = sorted_df[sorted_df['first_number'] == i]
    # Sort by 'second_number'
    filtered_df = filtered_df.sort_values('second_number')
    # Draw a bar chart
    filtered_df.plot.bar(x='second_number', y='use_nodes', rot=0, title=f'trace={i}', xlabel='Number of Tetants', ylabel='use_nodes')
    plt.show()

# Save to a new CSV file
# print(sorted_df)
# sorted_df.to_csv(out_file)


filter the node config according to 'memory' and 'cpu'

In [ ]:
import pandas as pd

# Load the dataframe from the CSV file
path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_node_list_all_node.csv'
df = pd.read_csv(path)

# Remove columns ['sn', 'gpu', 'model']
df = df.drop(columns=['sn', 'gpu', 'model'])

# Remove duplicate rows
df = df.drop_duplicates()

# Save to a new CSV file
df.to_csv('/home/fabing/projects/kubernetes-scheduler-simulator/simplify_node_list_all_node.csv', index=False)
print(df)


Uniformly sample the trace and generate the corresponding table

In [ ]:
import pandas as pd


nums = ['050', '100', '200', '250', '300']
for num in nums:
    path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_pod_list_cpu' + num + '.csv'
    df = pd.read_csv(path)
    df['num_gpu'] = 0
    df['gpu_milli'] = 0
    df['gpu_spec'] = ''
    running_df = df
    # Filter the dataframe
    # running_df = df[df['pod_phase'] == 'Running']
    # running_df = running_df[running_df['qos'] == 'LS']
    # running_df = running_df[running_df['num_gpu'] == 0]
    
    tenants = [1, 2, 3, 5, 10, 15, 20]
    for tenant in tenants:
        # Split the dataframe into parts
        dfs = [df.iloc[i::tenant].reset_index(drop=True) for i in range(tenant)]
        # Save each part to a separate CSV file
        for i, df_part in enumerate(dfs, 1):
            out_file = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/fabing_uniform_{}_{}_{}.csv'.format(num, tenant, i)
            print('data/fabing_uniform_{}_{}_{}'.format(num, tenant, i))
            df_part.to_csv(out_file.format(i), index=False)
            # print(df_part.head(2))


Add the price to the node list

In [ ]:
import pandas as pd

# Load the dataframe from the CSV file
path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_node_list_all_node.csv'
node_df = pd.read_csv(path)

# Load the price dataframe from the CSV file
path = '/home/fabing/projects/kubernetes-scheduler-simulator/node_list_price.csv'
price_df = pd.read_csv(path)
# Remove commas and convert to float
price_df['Aliyun cost (yuan/month)'] = price_df['Aliyun cost (yuan/month)'].replace(',', '', regex=True).astype(float)

result_df = pd.merge(node_df, price_df, on=['cpu_milli', 'memory_mib'], how='left')
result_df.to_csv('/home/fabing/projects/kubernetes-scheduler-simulator/node_list_all_node_price.csv', index=False)

In [ ]:
import pandas as pd

path = '/home/fabing/projects/kubernetes-scheduler-simulator/experiments/2023_cpu/fabing_cpu_050_1_1/05-BestFit/1.3/42/analysis_price.csv'
price_path = '/home/fabing/projects/kubernetes-scheduler-simulator/node_list_all_node_price.csv'

price_df = pd.read_csv(price_path)
total_cost = 0
with open(path, 'r') as f:
    line = f.readline()
    line = line.replace(r'\n', '')
    line = line.split(r',')
    for sn in line:
        sn = sn.strip()
        try:
            aliyun_cost = price_df.loc[price_df['sn'] == sn, 'Aliyun cost (yuan/month)']
            total_cost += aliyun_cost.values[0]
        except IndexError:
            print(f"No match found for sn: {sn}")
    print(total_cost)

Merge the node list and the table, and calulate the price

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataframe from the CSV file
path = '/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_price_discrete.csv'
out_file = '/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/uniform_analysis_price_discrete_sum.csv'
df = pd.read_csv(path)
# Extract the prefix of the 'workload' column
df['workload_prefix'] = df['workload'].str.rsplit('_', n=1).str[0]

# Calculate the average 'use_nodes' for each 'workload_prefix'
df = df.groupby('workload_prefix').sum()
df.drop(columns=['workload', 'sc_policy', 'tune'], inplace=True)
df.to_csv(out_file)
df = pd.read_csv(out_file)
# Split the 'workload_prefix' into four parts
df[['prefix', 'type', 'traces', 'tenants']] = df['workload_prefix'].str.split('_', expand=True)
# Convert the 'traces' and 'tenants' columns to numeric types
df[['traces', 'tenants']] = df[['traces', 'tenants']].apply(pd.to_numeric)
# df['cpu'] = df['cpu'].astype(float)
# df['mem'] = df['mem'].astype(float)
df['cpu'] = df['cpu'] / 1000
df['mem'] = df['mem'] / 1024
# Group by 'first_number' and calculate the mean of 'use_nodes'
# grouped_df = df.groupby('first_number')['cost'].mean()

# Sort by 'second_number'
sorted_df = df.sort_values('traces')
# Define color list
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

# Define bar width
bar_width = 0.15

# Create a new figure for 'cpu'
plt.figure()
# Loop over unique traces
for idx, i in enumerate(sorted_df['traces'].unique()):
    # Filter the dataframe
    filtered_df = sorted_df[sorted_df['traces'] == i]
    # Sort by 'tenants'
    filtered_df = filtered_df.sort_values('tenants')
    # Draw a bar chart for 'cpu' with offset
    plt.bar(filtered_df['tenants'] + idx * bar_width, filtered_df['cpu'], width=bar_width, color=colors[idx % len(colors)], label=f'cpu_trace={i}')
    # Add scatter plot and line
    # plt.scatter(filtered_df['tenants'] + idx * bar_width, filtered_df['cpu'], color=colors[idx % len(colors)])
# Set xticks
plt.xticks(sorted_df['tenants'].unique())
# Add a legend
plt.legend(loc='upper left')
# Show the plot
plt.show()

# Create a new figure for 'mem'
plt.figure()
# Loop over unique traces
for idx, i in enumerate(sorted_df['traces'].unique()):
    # Filter the dataframe
    filtered_df = sorted_df[sorted_df['traces'] == i]
    # Sort by 'tenants'
    filtered_df = filtered_df.sort_values('tenants')
    # Draw a bar chart for 'mem' with offset
    plt.bar(filtered_df['tenants'] + idx * bar_width, filtered_df['mem'], width=bar_width, color=colors[idx % len(colors)], label=f'mem_trace={i}')
# Set xticks
plt.xticks(sorted_df['tenants'].unique())
# Add a legend
plt.legend(loc='upper left')
# Show the plot
plt.show()

# Create a new figure for 'cost'
plt.figure()
# Loop over unique traces
for idx, i in enumerate(sorted_df['traces'].unique()):
    # Filter the dataframe
    filtered_df = sorted_df[sorted_df['traces'] == i]
    # Sort by 'tenants'
    filtered_df = filtered_df.sort_values('tenants')
    # Draw a bar chart for 'cost' with offset
    plt.bar(filtered_df['tenants'] + idx * bar_width, filtered_df['cost'], width=bar_width, color=colors[idx % len(colors)], label=f'cost_trace={i}')
# Set xticks
plt.xticks(sorted_df['tenants'].unique())
# Add a legend
plt.legend(loc='upper left')
# Show the plot
plt.show()

# save to a new CSV file
sorted_df.to_csv(out_file)

In [ ]:
# Draw bar charts
for i in sorted_df['traces'].unique():
    # Filter the dataframe
    filtered_df = sorted_df[sorted_df['traces'] == i]
    # Sort by 'second_number'
    filtered_df = filtered_df.sort_values('tenants')
    # Draw a bar chart for 'cpu'
    ax = filtered_df.plot.bar(x='tenants', y='cpu', rot=0, title=f'trace={i}', xlabel='Number of Tenants', ylabel='Usage of CPU')
    ax.legend(loc='upper left')
    plt.show()
    # Draw a bar chart for 'mem'
    ax = filtered_df.plot.bar(x='tenants', y='mem', rot=0, title=f'trace={i}', xlabel='Number of Tenants', ylabel='Usage of Memory')
    ax.legend(loc='upper left')
    plt.show()
    # Draw a bar chart for 'cost'
    ax = filtered_df.plot.bar(x='tenants', y='cost', rot=0, title=f'trace={i}', xlabel='Number of Tenants', ylabel='Cost')
    ax.legend(loc='upper left')
    plt.show()

extract timeline

In [ ]:
path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_pod_list_cpu050.csv'
data = pd.read_csv(path)

# average delete time - average create time
duration = data['deletion_time'].mean() - data['creation_time'].mean()
print(duration/60)
print(len(data)/(duration/60))
# 40 minutes for openstack 2700 events and 2100 RPC
# 444 minites for 10095 pods
2100/40

Resource usage of each tenant

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataframe from the CSV file
path = '/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_price_discrete.csv'
out_file = '/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/uniform_analysis_price_discrete_sum.csv'
df = pd.read_csv(path)
# Extract the prefix of the 'workload' column
df['workload_prefix'] = df['workload'].str.rsplit('_', n=1).str[0]

# Calculate the average 'use_nodes' for each 'workload_prefix'
df = df.groupby('workload_prefix').sum()
df.drop(columns=['workload', 'sc_policy', 'tune'], inplace=True)
df.to_csv(out_file)
df = pd.read_csv(out_file)
# Split the 'workload_prefix' into four parts
df[['prefix', 'type', 'traces', 'tenants']] = df['workload_prefix'].str.split('_', expand=True)
# Convert the 'traces' and 'tenants' columns to numeric types
df[['traces', 'tenants']] = df[['traces', 'tenants']].apply(pd.to_numeric)

# Sort by 'second_number'
sorted_df = df.sort_values('traces')
sorted_df.to_csv(out_file)
print('save to {}'.format(out_file))


save to /home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/uniform_analysis_price_discrete_sum.csv


In [9]:
import pandas as pd

traces = ['050', '100', '200', '250', '300']
for trace in traces:
    path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_pod_list_cpu{}.csv'.format(trace)
    data = pd.read_csv(path)

    # print sum of column cpu_milli and memory_mib
    print(trace)
    print(data['cpu_milli'].sum())
    print(data['memory_mib'].sum())


050
72761612
268502666
100
80292312
289071014
200
97253012
336263486
250
107460312
365787661
300
119444912
397746134
